## Importing xlsx

In [1]:
#%pip install openpyxl
#%pip install xlrd

In [2]:
import pandas as pd
import numpy as np
from apifunctions import *
import datetime

pd.set_option("display.max_rows", None, "display.max_columns", None)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/danigomezlechonbarrachina/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


In [3]:
wacc=pd.read_excel("../data/datos_Bloomberg/wacc.xlsx")
prices=pd.read_excel("../data/datos_Bloomberg/prices.xlsx")
sales_growth=pd.read_excel("../data/datos_Bloomberg/sales_growth.xlsx")
parameters=pd.read_excel("../data/datos_Bloomberg/parameters.xlsx")

In [4]:
company_list=wacc.ID

## ID's as index

In [5]:
datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.set_index("ID",inplace=True)

## Dealing with NANs

In [6]:
# We first want to delete the rows which have all NANs for the wacc, prices and sales_growth dataframes

datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.dropna(axis=0, how='all',inplace=True)   

#### For simplicity, in prices we are going to delete all the rows that have nans as that means the company was not trading at that point so it wont allow us to do the backtesting:

In [7]:
prices.dropna(axis=0,inplace=True)

#### Now we are going to delete the companies for which we dont have information in one of the three datasets

In [8]:
remove_columns(prices,wacc,sales_growth)
remove_columns(prices,wacc,sales_growth)

#### To deal with the rest of NANs in wacc and sales_growth we are going to fill them with the average of the values in its time range

In [9]:
fill_na_mean(wacc)
fill_na_mean(sales_growth)

#### At this point we have cleaned the three dataframes: wacc, prices, sales growth. Now we need to deal with the parameters one

In [10]:
final_company_list=wacc.index

In [11]:
deleted_companies=[]

for i in company_list:
    
    if i not in final_company_list:
        
        deleted_companies.append(i)

### Parameters dataframe

#### We first create column with the year only and we set the dates as index and transpose the dataframe to have it in the same format as the rest:

In [12]:
lista=[]

for i in parameters["Unnamed: 0"]:
    
    lista.append(str(i))

year=[]
for j in lista:
    year.append(j[:4])
    
    

In [13]:
parameters["dates"]=year
parameters_new=parameters.groupby("dates").sum()

In [14]:
parameters_new_t=parameters_new.transpose()

In [15]:
parameters_new_t["company_name"]=parameters_new_t.index

#### we add a column named company name with the ticker of each company

In [16]:
parameters_new_t["company_name"]=parameters_new_t["company_name"].str.replace('\.\d+', '')

#### As we dont have values for 2022 we drop the entire column

In [17]:
parameters_new_t=parameters_new_t.drop(["2022"],axis=1)

#### We convert the numbers to float and then delete the rows which have at least one missing value

In [18]:
#parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]].astype(dtype="float64")

In [19]:
parameters_new_t=parameters_new_t[~(parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]]==0).any(axis=1)]

#### Now we need to check for which companies we have full data

In [20]:
companies_we_have=parameters_new_t["company_name"]

In [21]:
final_comps=["IFF UN Equity ","NTAP UW Equity","ATO UN Equity","GWW UN Equity",     "UHS UN Equity ",     "WEC UN Equity ",     "SWKS UW Equity",     "HSY UN Equity ",     "DGX UN Equity",     "HRL UN Equity",     "EW UN Equity" ,      "MDLZ UW Equity" ,    "ATVI UW Equity",     "HUM UN Equity",      "ROK UN Equity",      "ITW UN Equity",     "TT UN Equity",       "IPG UN Equity",      "WM UN Equity",       "J UN Equity",        "BR UN Equity",       "GPC UN Equity",      "GIS UN Equity",      "GD UN Equity",       "KLAC UW Equity",     "COST UW Equity",     "PCAR UW Equity",     "GOOG UW Equity",     "AEE UN Equity",      "ECL UN Equity",      "EMR UN Equity",      "ODFL UW Equity",     "AON UN Equity",      "DVA UN Equity",      "CBRE UN Equity",     "MNST UW Equity",    "CTAS UW Equity",     "EL UN Equity",       "BRO UN Equity",      "AAPL UW Equity",     "AMGN UW Equity",     "KMX UN Equity",      "MA UN Equity",       "KMB UN Equity",     "PM UN Equity"  ,     "NDAQ UW Equity" ,    "SO UN Equity",   "BKNG UW Equity",  "MCO UN Equity", "OMC UN Equity","OKE UN Equity",      "PWR UN Equity",      "PH UN Equity",       "ROL UN Equity",      "PPL UN Equity",      "AME UN Equity",      "NOC UN Equity",     "RSG UN Equity" ,     "ISRG UW Equity",     "PPG UN Equity",      "CTSH UW Equity",     "HSIC UW Equity",     "RHI UN Equity",      "WST UN Equity",      "FDS UN Equity",      "FFIV UW Equity",     "ES UN Equity",       "UNH UN Equity",      "CHRW UW Equity",     "LLY UN Equity",      "APH UN Equity",      "ULTA UW Equity",     "ANTM UN Equity" ,    "LOW UN Equity",      "IEX UN Equity",      "MMC UN Equity",      "SPGI UN Equity",     "ACN UN Equity",      "AKAM UW Equity",     "CVS UN Equity",      "TXN UW Equity",      "SYY UN Equity",      "GOOGL UW Equity",    "TDG UN Equity",      "YUM UN Equity",      "NKE UN Equity",      "MKTX UW Equity",     "DOV UN Equity",      "ANSS UW Equity",     "DE UN Equity",       "PAYX UW Equity",     "ORLY UW Equity",     "BWA UN Equity",      "BIIB UW Equity",     "HON UW Equity ",    "WMT UN Equity",      "CSCO UW Equity",     "INTC UW Equity",     "MSFT UW Equity ",    "DG UN Equity",       "CI UN Equity",       "HAS UW Equity",      "AMCR UN Equity",    "UPS UN Equity",      "HCA UN Equity",      "TGT UN Equity",      "MHK UN Equity" ,     "FISV UW Equity",    "PG UN Equity",       "CNC UN Equity",      "APD UN Equity",      "AWK UN Equity",      "EQIX UW Equity",     "AOS UN Equity" ,     "CAT UN Equity",      "LKQ UW Equity",     "KO UN Equity",       "APTV UN Equity" ,    "INTU UW Equity",     "FLT UN Equity",      "IDXX UW Equity",     "ROST UW Equity",     "ROP UN Equity",      "HD UN Equity",       "MPWR UW Equity",    "QCOM UW Equity",     "MCD UN Equity",      "MRK UN Equity",      "MMM UN Equity",      "STE UN Equity" ,     "WBA UW Equity",      "CPRT UW Equity",     "FAST UW Equity",     "NDSN UW Equity",     "BBY UN Equity",     "NVR UN Equity",      "ALB UN Equity" ,     "EPAM UN Equity",     "BF/B UN Equity",     "CPB UN Equity",      "IPGP UW Equity",     "MTD UN Equity",      "RMD UN Equity",      
"TSCO UW Equity",     "AMAT UW Equity",     "CLX UN Equity",      "CMS UN Equity",      "WAB UN Equity",      "POOL UW Equity",     "CAG UN Equity",      "AZO UN Equity",      "DHR UN Equity",      "CMI UN Equity",      "EXPD UW Equity",     "ADM UN Equity",      "ADP UW Equity",      "VRSK UW Equity",     "CL UN Equity",       "LMT UN Equity",      "V UN Equity",        "ED UN Equity",       "MSCI UN Equity",     
"WAT UN Equity"]      

In [22]:
companies_to_use=[]

for i in final_comps:
    
    if i in final_company_list:
        
        companies_to_use.append(i)

In [23]:
wacc=wacc/100

In [24]:
sales_growth=sales_growth/100

In [25]:
parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["DATE"]=="EBITDA_TO_REVENUE"]=parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["company_name"]=="EBITDA_TO_REVENUE"]/100

In [26]:
parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]][parameters_new_t["DATE"]=="EBITDA_TO_REVENUE"]=parameters_new_t[["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]]/100

In [27]:
%store wacc
%store prices
%store sales_growth
%store parameters_new_t
%store companies_to_use

Stored 'wacc' (DataFrame)
Stored 'prices' (DataFrame)
Stored 'sales_growth' (DataFrame)
Stored 'parameters_new_t' (DataFrame)
Stored 'companies_to_use' (list)


In [31]:
prices

,2010-12-31 00:00:00,2011-12-30 00:00:00,2012-12-31 00:00:00,2013-12-31 00:00:00,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-30 00:00:00,2017-12-29 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,2020-12-31 00:00:00,2021-12-31 00:00:00
ID,,,,,,,,,,,,
LYB UN Equity,18.56,20.49,39.43,57.07,58.08,65.72,67.62,90.43,70.81,84.55,86.79,91.19
SBNY UW Equity,46.93,56.30,66.96,100.82,118.22,143.94,140.97,128.82,97.46,131.82,133.41,322.15
AXP UN Equity,36.36,40.60,50.17,80.19,83.13,63.00,68.41,93.19,90.75,120.25,118.88,162.73
VZ UN Equity,21.14,24.99,28.28,33.54,33.36,34.57,41.72,43.38,48.26,54.94,54.87,50.72
BA UN Equity,52.12,60.02,63.15,116.65,113.68,129.67,144.34,281.05,313.37,323.83,214.06,201.32
CAT UN Equity,68.77,67.77,68.82,71.20,73.68,56.75,80.65,141.16,116.37,139.08,176.59,204.76
JPM UN Equity,31.10,24.90,33.91,46.37,50.95,55.22,74.29,94.16,87.93,129.48,122.31,156.25
CVX UN Equity,58.02,69.79,73.32,87.48,81.36,68.20,93.21,103.08,93.03,107.24,79.41,116.13
KO UN Equity,23.19,25.38,27.04,31.70,33.37,35.08,34.96,39.99,42.69,51.49,52.76,58.76


In [29]:
companies_to_use

['ATO UN Equity',
 'GWW UN Equity',
 'SWKS UW Equity',
 'DGX UN Equity',
 'EW UN Equity',
 'ATVI UW Equity',
 'HUM UN Equity',
 'ROK UN Equity',
 'ITW UN Equity',
 'TT UN Equity',
 'IPG UN Equity',
 'WM UN Equity',
 'J UN Equity',
 'BR UN Equity',
 'GPC UN Equity',
 'GD UN Equity',
 'KLAC UW Equity',
 'PCAR UW Equity',
 'AEE UN Equity',
 'ECL UN Equity',
 'EMR UN Equity',
 'ODFL UW Equity',
 'AON UN Equity',
 'DVA UN Equity',
 'CBRE UN Equity',
 'MNST UW Equity',
 'EL UN Equity',
 'BRO UN Equity',
 'AAPL UW Equity',
 'AMGN UW Equity',
 'MA UN Equity',
 'KMB UN Equity',
 'PM UN Equity',
 'NDAQ UW Equity',
 'SO UN Equity',
 'BKNG UW Equity',
 'MCO UN Equity',
 'OMC UN Equity',
 'OKE UN Equity',
 'PWR UN Equity',
 'PH UN Equity',
 'ROL UN Equity',
 'PPL UN Equity',
 'AME UN Equity',
 'NOC UN Equity',
 'RSG UN Equity',
 'ISRG UW Equity',
 'PPG UN Equity',
 'CTSH UW Equity',
 'HSIC UW Equity',
 'RHI UN Equity',
 'WST UN Equity',
 'FFIV UW Equity',
 'ES UN Equity',
 'UNH UN Equity',
 'CHRW U